# Modélisation #7 XGBoost + HyperOpt

In [10]:
# Importation des packages

# Base
import importlib
import functions
importlib.reload(functions)
from functions import *

# Data Management
import pandas as pd
import numpy as np

# Modélisation
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Optimisation
from skopt import BayesSearchCV
from skopt.space import Real, Integer

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [11]:
# Importation des données

train_data = pd.read_csv('../data/Train_Data/7_train_data.csv', index_col=0)
train_scores = pd.read_csv('../data/Y_train_1rknArQ.csv', index_col=0)

sub_data = pd.read_csv('../data/Sub_Data/7_sub_data.csv', index_col=0)

train_data.head(5)

HOME_TEAM_SHOTS_TOTAL_season_sum  HOME_TEAM_SHOTS_INSIDEBOX_season_sum  \
ID                                                                           
0                                3.0                                   2.0   
1                                6.0                                   8.0   
2                                4.0                                   2.0   
3                                7.0                                   5.0   
4                                3.0                                   3.0   

    HOME_TEAM_SHOTS_OFF_TARGET_season_sum  \
ID                                          
0                                     5.0   
1                                     3.0   
2                                     5.0   
3                                     5.0   
4                                     2.0   

    HOME_TEAM_SHOTS_ON_TARGET_season_sum  \
ID                                         
0                                    2.0   
1                                    6.0   
2                                    2.0   
3                                    6.0   
4                                    3.0   

    HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum  HOME_TEAM_PASSES_season_sum  \
ID                                                                       
0                                     1.0                          2.0   
1                                     5.0                          8.0   
2                                     8.0                          1.0   
3                                     6.0                          9.0   
4                                     4.0                          4.0   

    HOME_TEAM_SUCCESSFUL_PASSES_season_sum  HOME_TEAM_SAVES_season_sum  \
ID                                                                       
0                                      2.0                         5.0   
1                                      7.0                        10.0   
2                                      1.0                         2.0   
3                                      9.0                         2.0   
4                                      3.0                         4.0   

    HOME_TEAM_CORNERS_season_sum  HOME_TEAM_FOULS_season_sum  \
ID                                                             
0                            3.0                         6.0   
1                            6.0                         8.0   
2                            2.0                         7.0   
3                            2.0                         0.0   
4                            4.0                         7.0   

    HOME_TEAM_YELLOWCARDS_season_sum  HOME_TEAM_REDCARDS_season_sum  \
ID                                                                    
0                                2.0                            2.0   
1                                3.0                            0.0   
2                                7.0                            6.0   
3                                0.0                            6.0   
4                                4.0                            6.0   

    HOME_TEAM_OFFSIDES_season_sum  HOME_TEAM_ATTACKS_season_sum  \
ID                                                                
0                             1.0                           2.0   
1                             7.0                           7.0   
2                             3.0                           4.0   
3                             8.0                           6.0   
4                             2.0                           4.0   

    HOME_TEAM_PENALTIES_season_sum  HOME_TEAM_SUBSTITUTIONS_season_sum  \
ID                                                                       
0                              6.0                                 8.0   
1                              6.0                                10.0   
2                              5.0                                10.0   
3      

In [12]:
# Je préfère être sûr que les indexs concordent. 

train_scores = train_scores.loc[train_data.index]

train_scores.head(5)

,HOME_WINS,DRAW,AWAY_WINS
ID,,,
0,0,0,1
1,0,1,0
2,0,0,1
3,1,0,0
4,0,1,0


In [13]:
# Préparation des données pour la prédiction. 

train_scores_1c = train_scores[['HOME_WINS', 'DRAW', 'AWAY_WINS']].idxmax(axis=1)
label_mapping = {'HOME_WINS': 0, 'DRAW': 1, 'AWAY_WINS': 2}
train_scores_1c = train_scores_1c.replace(label_mapping)

train_scores_1c.head(5)

C:\Users\Hugo\AppData\Local\Temp\ipykernel_12820\2909335398.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_scores_1c = train_scores_1c.replace(label_mapping)


ID
0    2
1    1
2    2
3    0
4    1
dtype: int64

In [14]:
x_train, x_test, y_train, y_test = train_test_split(train_data, train_scores_1c, train_size=0.9, random_state=42)

XGB MODEL

In [15]:
# model = xgb.XGBClassifier(
#     objective='multi:softmax',
#     num_class=3,
# )

HyperOpt

In [16]:
# Je définis l'espace de recherche pour les hyperparamètres de XGBoost

param_space = {
    'max_depth': Integer(4, 12),  # Profondeur des arbres
    'learning_rate': Real(0.005, 0.1, prior='log-uniform'),  # Taux d'apprentissage
    'n_estimators': Integer(200, 1500),  # Nombre d'arbres
    'gamma': Real(0, 0.3),  # Minimum loss reduction
    'colsample_bytree': Real(0.5, 1.0),  # Fraction de colonnes à utiliser pour chaque arbre
    'subsample': Real(0.5, 1.0),  # Fraction de données à utiliser pour chaque arbre
    'reg_alpha': Real(0.01, 0.5, prior='log-uniform'),  # L1 regularization
    'reg_lambda': Real(0.01, 0.5, prior='log-uniform'),  # L2 regularization
    'min_child_weight': Integer(1, 6)  # Minimum de poids d'instances pour une feuille
}


In [17]:
# La fonction que l'on va minimiser
    
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Prédiction de classes multiples
    num_class=3  # Trois classes : HOME_WINS, DRAW, AWAY_WINS
)

opt = BayesSearchCV(
    estimator=xgb_model,
    search_spaces=param_space,
    n_iter=50,  # Nombre total d'itérations (essais d'hyperparamètres)
    cv=4,  # Cross-validation à 3 plis
    n_jobs=-1,  # Utiliser tous les cœurs disponibles
    verbose=1,
    scoring='accuracy',  # Optimisation sur la précision
    random_state=42
)

In [18]:
opt.fit(x_train, y_train)

print("Meilleurs hyperparamètres trouvés :", opt.best_params_)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fi

In [19]:
y_pred_final = opt.predict(x_test)

final_accuracy = accuracy_score(y_test, y_pred_final)
print(f"Accuracy du modèle optimisé : {final_accuracy * 100:.2f}%")

Accuracy du modèle optimisé : 51.58%


# Prédictions

In [20]:
y_sub = opt.predict(sub_data)

In [21]:
# Reconstruction du DataFrame y_sub avec les colonnes 'HOME_WINS', 'DRAW', 'AWAY_WINS'
y_sub_df = pd.DataFrame(y_sub, columns=['PRED'])

# Transformation en one-hot encoding pour obtenir les 3 colonnes
y_sub_df['HOME_WINS'] = (y_sub_df['PRED'] == 0).astype(int)
y_sub_df['DRAW'] = (y_sub_df['PRED'] == 1).astype(int)
y_sub_df['AWAY_WINS'] = (y_sub_df['PRED'] == 2).astype(int)

# Ajout de la colonne 'ID' depuis sub_data
y_sub_df['ID'] = sub_data.index

# Suppression de la colonne 'PRED'
y_sub_df.drop('PRED', axis=1, inplace=True)

# Réorganiser les colonnes pour avoir 'ID' en premier
y_sub_df = y_sub_df[['ID', 'HOME_WINS', 'DRAW', 'AWAY_WINS']]

# Afficher les premières lignes du résultat final
y_sub_df = y_sub_df.set_index('ID')
y_sub_df.head()

,HOME_WINS,DRAW,AWAY_WINS
ID,,,
12303,1,0,0
12304,0,0,1
12305,1,0,0
12306,1,0,0
12307,1,0,0


In [22]:
y_sub_df.to_csv('../data/Sub_Data/19_sub_scores.csv')